In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [6]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-data_test.pkl',
 'molecule-with-openbabel_test.pkl',
 'eachtype_te

In [ ]:
mytrial = []

In [ ]:
"""
y=sd, type=0
287
{'columns': ['tertiary_angle_4', 'adN1', 'vander_H.y', 'tertiary_distance_0', 'tertiary_angle_5', 'dist_C_2_y', 'dist_C_4_y', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_H_2_x', 'dist_O_1_y', 'tertiary_distance_8', 'molecule_dist_min', 'atom_index_1_cycle_size_mean', 'molecule_atom_index_1_dist_min_diff', 'adC2', 'dist_H_4_y', 'distC1', 'dist_to_type_0_mean', 'dist_C_2_x', 'dist_to_type_mean', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_O_0_y', 'distance_farthest_0', 'molecule_atom_index_0_dist_max_div', 'dist_H_2_y', 'inv_distPR', 'vander_O.x', 'tertiary_angle_3', 'cos_f0', 'adC1', 'molecule_type_dist_std_diff', 'cos_center0', 'eem_1', 'molecule_atom_1_dist_mean', 'dist_C_1_y', 'dist_H_0_y', 'dist_O_1_x', 'tertiary_atom_4', 'dist_C_3_y', 'dist_H_1_x', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_4', 'dist_to_type_std', 'yukawa_H.y', 'atom_1_n_bonds', 'dist_C_0_x', 'molecule_type_dist_mean_diff', 'dist_xyz', 'atom_1_bond_lengths_max', 'cos_center1', 'dist_N_1_y', 'inv_dist1R', 'dist_O_0_x', 'distC0', 'molecule_couples', 'tertiary_angle_6', 'inv_dist0R', 'tertiary_atom_2', 'dist_to_type_1_mean', 'dist_C_3_x', 'coulomb_H.x', 'tertiary_angle_1', 'dist_N_0_y', 'tertiary_distance_3', 'tertiary_atom_3', 'tertiary_distance_1', 'molecule_dist_max', 'dist_C_0_y', 'dist_H_3_x', 'NH', 'dist_H_3_y', 'atom_index_farthest_0', 'eem_0', 'dist_C_1_x', 'tertiary_angle_2', 'inv_distPE', 'tertiary_distance_2', 'dist_C_4_x', 'tertiary_atom_1', 'mean_molecule_atom_1_dist_xyz', 'molecule_dist_mean', 'max_molecule_atom_0_dist_xyz', 'inv_dist1E', 'bond_atom', 'dist_H_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=1
377
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=2
467
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=3
754
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=4
862
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=5
970
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=6
197
{'columns': ['cos_c0', 'molecule_atom_index_0_dist_min_diff', 'dist_C_1_x', 'molecule_atom_index_0_dist_std_diff', 'dist_N_0_y', 'dist_to_type_mean', 'tertiary_angle_4', 'max_molecule_atom_1_dist_xyz', 'tertiary_atom_0', 'tertiary_distance_3', 'dist_to_type_0_mean', 'dist_C_0_x', 'tertiary_distance_1', 'distance_c1', 'vander_C.y', 'tertiary_angle_1', 'dist_C_1_y', 'dist_O_0_y', 'cos_f1', 'dist_H_0_x', 'adC2', 'dist_H_0_y', 'molecule_atom_index_0_dist_std_div', 'tertiary_atom_2', 'cos_c1', 'dist_xyz', 'inv_dist1R', 'min_molecule_atom_0_dist_xyz', 'dist_to_type_std', 'tertiary_distance_4', 'tertiary_atom_1', 'dist_C_0_y', 'tertiary_angle_0', 'inv_dist0R', 'inv_distPE', 'vander_C.x', 'eem_1', 'cos_f0_f1', 'distance_farthest_0', 'molecule_atom_1_dist_min_diff', 'cos_f0', 'tertiary_angle_2', 'adC3', 'dist_O_0_x', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_H_1_y', 'atom_1_bond_lengths_mean', 'tertiary_angle_3', 'distH0', 'dist_C_2_y', 'adC1', 'dist_to_type_1_mean', 'tertiary_distance_2', 'max_distance_y', 'cos_c0_c1', 'dist_C_2_x', 'dist_C_3_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=sd, type=7
119
{'columns': ['tertiary_distance_3', 'tertiary_atom_1', 'dist_C_4_x', 'bond_atom', 'atom_index_1_cycle_size_mean', 'tertiary_angle_3', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distN0', 'cos_c0_c1', 'dist_to_type_mean', 'atom_1_bond_lengths_min', 'dist_N_1_x', 'dist_O_0_y', 'adC2', 'molecule_atom_index_1_dist_min_div', 'cos_c0', 'dist_C_1_x', 'tertiary_angle_2', 'dist_to_type_std', 'dist_C_2_y', 'atom_1_bond_lengths_max', 'tertiary_atom_3', 'cos_f0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_1_y', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_distance_1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_distance_2', 'dist_C_0_x', 'eem_1', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_std', 'inv_dist0R', 'inv_distPR', 'dist_N_0_y', 'atom_1_n_bonds', 'tertiary_angle_1', 'dist_C_0_y', 'inv_dist1R', 'dist_C_3_x', 'tertiary_angle_0', 'cos_c1', 'dist_to_type_0_mean', 'adC1', 'molecule_atom_index_1_dist_min_diff', 'molecule_type_0_dist_std_diff'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
"""



In [6]:
# 'scalar_coupling_constant', 'fc', 'sd','pso','dso'
for y in ['sd']:
    for t in np.arange(6,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
        break
    

y=sd, type=6
197
{'columns': ['cos_c0', 'molecule_atom_index_0_dist_min_diff', 'dist_C_1_x', 'molecule_atom_index_0_dist_std_diff', 'dist_N_0_y', 'dist_to_type_mean', 'tertiary_angle_4', 'max_molecule_atom_1_dist_xyz', 'tertiary_atom_0', 'tertiary_distance_3', 'dist_to_type_0_mean', 'dist_C_0_x', 'tertiary_distance_1', 'distance_c1', 'vander_C.y', 'tertiary_angle_1', 'dist_C_1_y', 'dist_O_0_y', 'cos_f1', 'dist_H_0_x', 'adC2', 'dist_H_0_y', 'molecule_atom_index_0_dist_std_div', 'tertiary_atom_2', 'cos_c1', 'dist_xyz', 'inv_dist1R', 'min_molecule_atom_0_dist_xyz', 'dist_to_type_std', 'tertiary_distance_4', 'tertiary_atom_1', 'dist_C_0_y', 'tertiary_angle_0', 'inv_dist0R', 'inv_distPE', 'vander_C.x', 'eem_1', 'cos_f0_f1', 'distance_farthest_0', 'molecule_atom_1_dist_min_diff', 'cos_f0', 'tertiary_angle_2', 'adC3', 'dist_O_0_x', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_H_1_y', 'atom_1_bond_lengths_mean', 'tertiary_angle_3', 'distH0', 'dist_C_2_y', 'adC1', 'dist_to_type_1_m

In [7]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['sd']

In [8]:
columns_list = [{'columns': ['tertiary_angle_4', 'adN1', 'vander_H.y', 'tertiary_distance_0', 'tertiary_angle_5', 'dist_C_2_y', 'dist_C_4_y', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_H_2_x', 'dist_O_1_y', 'tertiary_distance_8', 'molecule_dist_min', 'atom_index_1_cycle_size_mean', 'molecule_atom_index_1_dist_min_diff', 'adC2', 'dist_H_4_y', 'distC1', 'dist_to_type_0_mean', 'dist_C_2_x', 'dist_to_type_mean', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_O_0_y', 'distance_farthest_0', 'molecule_atom_index_0_dist_max_div', 'dist_H_2_y', 'inv_distPR', 'vander_O.x', 'tertiary_angle_3', 'cos_f0', 'adC1', 'molecule_type_dist_std_diff', 'cos_center0', 'eem_1', 'molecule_atom_1_dist_mean', 'dist_C_1_y', 'dist_H_0_y', 'dist_O_1_x', 'tertiary_atom_4', 'dist_C_3_y', 'dist_H_1_x', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_4', 'dist_to_type_std', 'yukawa_H.y', 'atom_1_n_bonds', 'dist_C_0_x', 'molecule_type_dist_mean_diff', 'dist_xyz', 'atom_1_bond_lengths_max', 'cos_center1', 'dist_N_1_y', 'inv_dist1R', 'dist_O_0_x', 'distC0', 'molecule_couples', 'tertiary_angle_6', 'inv_dist0R', 'tertiary_atom_2', 'dist_to_type_1_mean', 'dist_C_3_x', 'coulomb_H.x', 'tertiary_angle_1', 'dist_N_0_y', 'tertiary_distance_3', 'tertiary_atom_3', 'tertiary_distance_1', 'molecule_dist_max', 'dist_C_0_y', 'dist_H_3_x', 'NH', 'dist_H_3_y', 'atom_index_farthest_0', 'eem_0', 'dist_C_1_x', 'tertiary_angle_2', 'inv_distPE', 'tertiary_distance_2', 'dist_C_4_x', 'tertiary_atom_1', 'mean_molecule_atom_1_dist_xyz', 'molecule_dist_mean', 'max_molecule_atom_0_dist_xyz', 'inv_dist1E', 'bond_atom', 'dist_H_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['cos_c0', 'molecule_atom_index_0_dist_min_diff', 'dist_C_1_x', 'molecule_atom_index_0_dist_std_diff', 'dist_N_0_y', 'dist_to_type_mean', 'tertiary_angle_4', 'max_molecule_atom_1_dist_xyz', 'tertiary_atom_0', 'tertiary_distance_3', 'dist_to_type_0_mean', 'dist_C_0_x', 'tertiary_distance_1', 'distance_c1', 'vander_C.y', 'tertiary_angle_1', 'dist_C_1_y', 'dist_O_0_y', 'cos_f1', 'dist_H_0_x', 'adC2', 'dist_H_0_y', 'molecule_atom_index_0_dist_std_div', 'tertiary_atom_2', 'cos_c1', 'dist_xyz', 'inv_dist1R', 'min_molecule_atom_0_dist_xyz', 'dist_to_type_std', 'tertiary_distance_4', 'tertiary_atom_1', 'dist_C_0_y', 'tertiary_angle_0', 'inv_dist0R', 'inv_distPE', 'vander_C.x', 'eem_1', 'cos_f0_f1', 'distance_farthest_0', 'molecule_atom_1_dist_min_diff', 'cos_f0', 'tertiary_angle_2', 'adC3', 'dist_O_0_x', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_H_1_y', 'atom_1_bond_lengths_mean', 'tertiary_angle_3', 'distH0', 'dist_C_2_y', 'adC1', 'dist_to_type_1_mean', 'tertiary_distance_2', 'max_distance_y', 'cos_c0_c1', 'dist_C_2_x', 'dist_C_3_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_3', 'tertiary_atom_1', 'dist_C_4_x', 'bond_atom', 'atom_index_1_cycle_size_mean', 'tertiary_angle_3', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distN0', 'cos_c0_c1', 'dist_to_type_mean', 'atom_1_bond_lengths_min', 'dist_N_1_x', 'dist_O_0_y', 'adC2', 'molecule_atom_index_1_dist_min_div', 'cos_c0', 'dist_C_1_x', 'tertiary_angle_2', 'dist_to_type_std', 'dist_C_2_y', 'atom_1_bond_lengths_max', 'tertiary_atom_3', 'cos_f0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_1_y', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_distance_1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_distance_2', 'dist_C_0_x', 'eem_1', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_std', 'inv_dist0R', 'inv_distPR', 'dist_N_0_y', 'atom_1_n_bonds', 'tertiary_angle_1', 'dist_C_0_y', 'inv_dist1R', 'dist_C_3_x', 'tertiary_angle_0', 'cos_c1', 'dist_to_type_0_mean', 'adC1', 'molecule_atom_index_1_dist_min_diff', 'molecule_type_0_dist_std_diff'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [9]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [10]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-18 08:26:09.606890,modeling for 0,88,0.004685,0.005375,0.000690,modeling for 0
1,2019-07-18 08:26:51.963880,modeling for 3,120,0.001748,0.002186,0.000439,modeling for 3
2,2019-07-18 08:26:55.003408,modeling for 1,70,0.002421,0.003730,0.001309,modeling for 1
3,2019-07-18 08:27:06.924764,modeling for 4,120,0.002044,0.003259,0.001215,modeling for 4
4,2019-07-18 08:28:11.274327,modeling for 2,70,0.006842,0.007488,0.000645,modeling for 2
5,2019-07-18 08:28:41.668152,modeling for 6,58,0.002394,0.002716,0.000321,modeling for 6
6,2019-07-18 08:30:58.213068,modeling for 5,120,0.005233,0.005649,0.000415,modeling for 5
7,2019-07-18 08:31:06.980405,modeling for 7,49,0.002414,0.003231,0.000818,modeling for 7


In [11]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.005239009795675559
0 0.005375204370837576
3 0.002186181296557227
1 0.0037304742788296943
4 0.0032589139294807246
2 0.007487543498037491
6 0.002715714355614683
5 0.005648543233534113
7 0.003231371470466261


In [13]:
print(df_valid_pred.shape)
df_valid_pred.tail()

(4658147, 3)


,index,predict,fold_n
4658142,4658142,0.032174,1.0
4658143,4658143,0.006078,2.0
4658144,4658144,-0.024607,3.0
4658145,4658145,0.130370,2.0
4658146,4658146,0.368041,1.0


In [12]:
print(df_test_pred.shape)
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.096841,0.110271,0.094608,0.097868,0.091781
2505538,7163685,0.048413,0.051885,0.072798,0.087132,0.098942
2505539,7163686,-0.029385,-0.014471,-0.022175,-0.025193,-0.029875
2505540,7163687,0.155941,0.130100,0.122157,0.130235,0.160987
2505541,7163688,0.394770,0.383084,0.384197,0.395634,0.383516


In [17]:
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['sd'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_pickle('../../data/oof/lgbm_sd_test.pkl')

In [18]:
df_submit = df_valid_pred.rename(columns={'predict':'sd', 'index':'id'})
df_submit.to_pickle('../../data/oof/lgbm_sd_train.pkl')